# Exercises - Fixed-Income Options


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


# 1. Black's Formula for Bond Options


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import erf, log, sqrt

FILE = 'discount_curve_2025-02-13.xlsx'
curve = pd.read_excel(FILE, sheet_name='discount curve')
curve['maturity date'] = pd.to_datetime(curve['maturity date'])
curve = curve.sort_values('ttm')

t_grid = curve['ttm'].astype(float).to_numpy()
Z_grid = curve['discount'].astype(float).to_numpy()

def Z0(t):
    return float(np.interp(t, t_grid, Z_grid))

## 1.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `annual` frequency
* annualized coupon rate of `cpn=6%`.

Use the bond-pricing formula along with the discount rates in the data file to price this bond.


In [14]:
T = 3
N = 100 #face value
FREQ = 1 #annual frequency
CPN = 0.06 #6% coupon rate

#Valuation Date inferred from first cash flow and first discount factor
val_date = curve['maturity date'].iloc[0] - pd.to_timedelta(t_grid[0]*365, unit='D')

pay_times = np.arange(1, T+1)
coupon = N * CPN / FREQ
cfs = np.full(T, coupon, dtype=float)
cfs[-1] += N # add face value to the last cash flow

P0 = np.sum(cfs * np.array([Z0(t) for t in pay_times]))

print(f"Vanilla Bond Price: ${P0:.2f} on Valuation Date: {val_date.date()}")

Vanilla Bond Price: $104.99 on Valuation Date: 2025-02-11


## 1.2.

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`
* vol of `2.68%`
* forward price of `103.31.`

What is the value of the issuer's call option?


In [18]:
t = 0.0
Topt = 1.5
TAU = Topt - t
K = 100.0
sigma = 0.0268
F_t = 103.31

def N(x): 
    "Normal CDF"
    return 0.5 * (1 + erf(x / sqrt(2)))

#Interpolate discount factor to option expiry Tau
Z_0T = Z0(TAU)

#Apply Black's Formula
d1 = (log(F_t / K) + 0.5 * sigma ** 2 * TAU) / (sigma * sqrt(TAU))
d2 = d1 - sigma * sqrt(TAU)

c_0 = Z_0T * (F_t * N(d1) - K * N(d2))
print(f'Z(0, {TAU}) = {Z_0T:.4f}')
print(f'd1 = {d1:.4f}, d2 = {d2:.4f}')
print(f'Issuer Call Option Value = ${c_0:.2f}')

Z(0, 1.5) = 0.9392
d1 = 1.0085, d2 = 0.9757
Issuer Call Option Value = $3.37


## 1.3.

What is the price of the callable bond? 

The **callable** bond is the bond issued with an embedded call option (long the issuer.) Thus, it is the value of the vanilla bond minus the value of the call option.


In [17]:
call_value = c_0

P_callable = P0 - call_value
print(f'Callable Bond Price = ${P_callable:.2f}')

Callable Bond Price = $101.61


## 1.4.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?


By pricing the option on the forward and switching to the T-forward measure using the zero-coupon bond as numeraire, all coupons/dividends are absorbed into the forward price, the drift dissappears, and Black's formula applies directly. With Black-Scholes, we would need to model stochastic interest rates. 

## 1.5

Redo 1.2. Suppose the market prices the call option at `3.50`.

Solve for the implied volatility.


In [19]:
call_mkt = 3.5

def black_call_price(sigma):
    d1 = (log(F_t / K) + 0.5 * sigma ** 2 * TAU) / (sigma * sqrt(TAU))
    d2 = d1 - sigma * sqrt(TAU)
    return Z_0T * (F_t * N(d1) - K * N(d2))

def objective(sigma):
    "Objective: model price - market price"
    return black_call_price(sigma) - call_mkt

#Solve for implied vol: sigma_bond ~ D * sigma_rate * r

low, high = 1e-6, 0.50 #0% to 50% vol
for _ in range(100):
    mid = 0.5 * (low + high)
    if objective(mid) > 0:
        high = mid
    else:
        low = mid
implied_vol = mid

print(f'Implied Volatility = {implied_vol:.4%}')


Implied Volatility = 3.0941%


In [20]:
from scipy.optimize import fsolve
def black_implied_vol(F, K, tau, Z, call_mkt, volGuess=0.03):
    func = lambda sigma: black_call_price(sigma) - call_mkt
    sig_star = fsolve(func, volGuess)[0]
    return sig_star

sig = black_implied_vol(F_t, K, TAU, Z_0T, call_mkt)
print(f'Implied Volatility (scipy) = {sig:.4%}')

Implied Volatility (scipy) = 3.0941%


/var/folders/1_/gb2cs27x3nq4pp7f6whksw7h0000gn/T/ipykernel_26214/228822721.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 0.5 * (1 + erf(x / sqrt(2)))
